In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from collections import defaultdict 
import jax
import flax
import chex
from jaxtyping import ArrayLike
from typing import Union, TypeVar
import numpy as np
import matplotlib.pyplot as plt

from tracr.compiler.validating import validate
from tracr.rasp.rasp import Map, SequenceMap, LinearSequenceMap, Select, Aggregate, Comparison, SelectorWidth, indices, tokens 
from tracr.rasp import rasp
from tracr.compiler import compiling
from tracr.compiler.assemble import AssembledTransformerModel
from tracr.compiler.craft_model_to_transformer import NoTokensError
from tracr.compiler.basis_inference import InvalidValueSetError
from tracr.compiler import rasp_to_graph

from rasp_tokenizer import tokenizer
from rasp_tokenizer import vocab
from rasp_tokenizer.compiling import COMPILER_BOS
from rasp_generator.utils import sample_test_input, FunctionWithRepr, annotate_type
from rasp_generator import sampling, utils, map_primitives
from rasp_tokenizer import paths
from rasp_tokenizer import vocab, data_utils, lib


rng = np.random.default_rng(0)

In [2]:
def sort():
    smaller = Select(tokens, tokens, rasp.Comparison.LT)
    target = SelectorWidth(smaller)
    sel_new = Select(target, indices, rasp.Comparison.EQ)
    return Aggregate(sel_new, tokens)

In [7]:
m, toks, _ = tokenizer.compile_and_tokenize(sort())

2024-02-02 10:06:44.537691: W external/xla/xla/service/gpu/nvptx_compiler.cc:698] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [9]:
{
    k: tokenizer.decode(v) for k, v in toks.items()
}

{'layer_0/attn': ['BOS',
  'sop_1',
  'categorical',
  'SelectorWidth',
  'tokens',
  'tokens',
  'LT',
  'EOS'],
 'layer_0/mlp': ['BOS', 'EOS'],
 'layer_1/attn': ['BOS',
  'sop_0',
  'categorical',
  'SelectAggregate',
  'sop_1',
  'indices',
  'EQ',
  'tokens',
  'EOS'],
 'layer_1/mlp': ['BOS', 'EOS']}

In [6]:
s = sort()
s([4,3,2,3])

[2, 3.0, None, 4]